In [44]:
from datetime import datetime, date
from pyspark.sql import SparkSession, Row, functions as F
from time import perf_counter
import pandas as pd
import logging

# type hint thingies
from typing import Iterator

In [25]:
# Spark session & context

# create it in local mode
spark = SparkSession.builder.master("local").getOrCreate()
sc = spark.sparkContext
spark.conf.set('spark.sql.repl.eagerEval.enabled', False)

In [14]:
# Sum of the first 100 whole numbers
rdd = sc.parallelize(range(200 + 1))

start = perf_counter()
print("Sum:", rdd.sum())
print(f"Took {perf_counter()-start}ms")

Sum: 20100
Took 0.1385273230043822ms


In [15]:
# Creates dataframe. No explicit schema specified - will sample row to get schema instead
# Alternatively can pass in `schema` param
# Dataframe can be created from pandas DF as well
df = spark.createDataFrame([
    Row(a=1, b=2., c='string1', d=date(2000, 1, 1), e=datetime(2000, 1, 1, 12, 0)),
    Row(a=2, b=3., c='string2', d=date(2000, 2, 1), e=datetime(2000, 1, 2, 12, 0)),
    Row(a=4, b=5., c='string3', d=date(2000, 3, 1), e=datetime(2000, 1, 3, 12, 0))
])
df

DataFrame[a: bigint, b: double, c: string, d: date, e: timestamp]

In [16]:
# Pandaas dataframe is created col by col
pandas_df = pd.DataFrame({
    'a': [1, 2, 3],
    'b': [2., 3., 4.],
    'c': ['string1', 'string2', 'string3'],
    'd': [date(2000, 1, 1), date(2000, 2, 1), date(2000, 3, 1)],
    'e': [datetime(2000, 1, 1, 12, 0), datetime(2000, 1, 2, 12, 0), datetime(2000, 1, 3, 12, 0)]
})
df = spark.createDataFrame(pandas_df)
df

DataFrame[a: bigint, b: double, c: string, d: date, e: timestamp]

In [17]:
df.show()
df.printSchema()

+---+---+-------+----------+-------------------+
|  a|  b|      c|         d|                  e|
+---+---+-------+----------+-------------------+
|  1|2.0|string1|2000-01-01|2000-01-01 12:00:00|
|  2|3.0|string2|2000-02-01|2000-01-02 12:00:00|
|  3|4.0|string3|2000-03-01|2000-01-03 12:00:00|
+---+---+-------+----------+-------------------+

root
 |-- a: long (nullable = true)
 |-- b: double (nullable = true)
 |-- c: string (nullable = true)
 |-- d: date (nullable = true)
 |-- e: timestamp (nullable = true)



In [18]:
# eager evaluation - displays like this in jupyter
# spark.conf.set('spark.sql.repl.eagerEval.enabled', True)
# df

a,b,c,d,e
1,2.0,string1,2000-01-01,2000-01-01 12:00:00
2,3.0,string2,2000-02-01,2000-01-02 12:00:00
3,4.0,string3,2000-03-01,2000-01-03 12:00:00


In [23]:
# Configure logging - find non-worker logs in docker container
# logging.basicConfig(level=logging.INFO, filename="log.txt")
# logger = logging.getLogger(__name__)
# logger.info("test from outside")

def pandas_filter_func(iterator: Iterator[pd.DataFrame]) -> Iterator[pd.DataFrame]:
    # logger.info("test from func") # not sure how to see these
    for pandas_df in iterator:
        yield pandas_df[pandas_df.a == 1]

df.mapInPandas(pandas_filter_func, schema=df.schema).show()

+---+---+-------+----------+-------------------+
|  a|  b|      c|         d|                  e|
+---+---+-------+----------+-------------------+
|  1|2.0|string1|2000-01-01|2000-01-01 12:00:00|
+---+---+-------+----------+-------------------+



In [20]:
sc.uiWebUrl

'http://8fd66782d827:4040'

In [27]:
df2 = spark.createDataFrame([
    Row('don','baltimore',12),
    Row('jerry','boston',19),
    Row('bob','baltimore',99),
    Row('cameron','baltimore',13),
    Row('james','seattle',1),
    Row('peter','seattle',2),
], schema = 'name: string, city: string, id: long')

df2

DataFrame[name: string, city: string, id: bigint]

In [47]:
df2.groupBy('city').count().show()

df2.groupby("city").agg({'name': 'collect_set'}).show()

df2.groupby("city").agg(F.collect_set('name')).show()

+---------+-----+
|     city|count|
+---------+-----+
|   boston|    1|
|  seattle|    2|
|baltimore|    3|
+---------+-----+

+---------+-------------------+
|     city|  collect_set(name)|
+---------+-------------------+
|   boston|            [jerry]|
|  seattle|     [peter, james]|
|baltimore|[bob, cameron, don]|
+---------+-------------------+

+---------+-------------------+
|     city|  collect_set(name)|
+---------+-------------------+
|   boston|            [jerry]|
|  seattle|     [peter, james]|
|baltimore|[bob, cameron, don]|
+---------+-------------------+

